# In-class exercise 9: Deep Learning (Part 1A)
In this notebook we will see how to write efficient and numerically stable code.

In [42]:
import numpy as np
import matplotlib.pyplot as plt
import time

%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import minmax_scale

In [43]:
X, y = load_breast_cancer(return_X_y=True)

# Scale each feature to [-1, 1] range
X = minmax_scale(X, feature_range=(-1, 1))

In [44]:
X.shape, y.shape

((569, 30), (569,))

# 1. Vectorization

## 1.1. Logistic regression (two classes)

**Setting:** Logistic regression (two classes)

**Task:** Generate predictions for the entire dataset

In [45]:
n_features = X.shape[1]
w = np.random.normal(size=[n_features], scale=0.1)  # weight vector
b = np.random.normal(size=[1])  # bias

In [46]:
w, b

(array([-0.0730886 , -0.0687235 , -0.02745307,  0.04856755, -0.0440173 ,
        -0.05506874, -0.0837222 , -0.06412004,  0.03203243, -0.19540539,
         0.00199596, -0.04906799,  0.12444059,  0.06578282,  0.12782646,
        -0.08041956, -0.08792368, -0.07675442, -0.06178841, -0.09958009,
         0.04062385,  0.06327119,  0.07949416,  0.09012991, -0.02262698,
        -0.05700673,  0.07962108, -0.01429931, -0.21616397,  0.10368435]),
 array([-1.66105378]))

In [47]:
def sigmoid(t):
    """Apply sigmoid to the input array."""
    return 1 / (1 + np.exp(-t))

### Bad - for loops

In [48]:
n_samples = X.shape[0]
n_samples

569

In [49]:
def predict_for_loop(X, w, b):
    """Generate predictions with a logistic regression model using a for-loop.
    
    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)
        
    Returns:
        y: probabilies of the positive class, shape (N)
    """
    n_samples = X.shape[0]
    y = np.zeros([n_samples])
    for i in range(n_samples):
        score = np.dot(X[i], w) + b
        y[i] = sigmoid(score)
    return y

### Good - vectorization

In [50]:
def predict_vectorized(X, w, b):
    """Generate predictions with a logistic regression model using vectorized operations.
    
    Args:
        X: data matrix, shape (N, D)
        w: weights vector, shape (D)
        b: bias term, shape (1)
        
    Returns:
        y: probabilies of the positive class, shape (N)
    """
    scores = X @ w + b
    y = sigmoid(scores)
    return y

### Compare the runtime of two variants

In [51]:
%%timeit
predict_for_loop(X, w, b)

3.35 ms ± 88.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [52]:
%%timeit
predict_vectorized(X, w, b)

14.1 µs ± 71.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## 1.2. K-nearest neighbors
A more complicated task: compute the matrix of pairwise distances.

Given a data matrix `X` of size `[N, D]`, compute the matrix `dist` of pairwise distances of size `[N, N]`, where `dist[i, j] = l2_distance(X[i], X[j])`.

### Bad - for loops

In [53]:
def l2_distance(x, y):
    """Compute Euclidean distance between two vectors."""
    return np.sqrt(np.sum((x - y) ** 2))

In [54]:
def distances_for_loop(X):
    """Compute pairwise distances between all instances (for loop version).
    
    Args:
        X: data matrix, shape (N, D)
        
    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    n_samples = X.shape[0]
    distances = np.zeros([n_samples, n_samples])
    for i in range(n_samples):
        for j in range(n_samples):
            distances[i, j] = l2_distance(X[i], X[j])
    return distances

In [55]:
dist1 = distances_for_loop(X)

### Good - vectorization

How can we compute all the distances in a vectorized way?

Start with a simpler example.

In [56]:
x = np.arange(5, dtype=np.float64)

In [57]:
x.reshape([-1,1]) - x.reshape([1,-1]) 

array([[ 0., -1., -2., -3., -4.],
       [ 1.,  0., -1., -2., -3.],
       [ 2.,  1.,  0., -1., -2.],
       [ 3.,  2.,  1.,  0., -1.],
       [ 4.,  3.,  2.,  1.,  0.]])

Increase the dimension of an array using `np.newaxis`

In [58]:
x[:, None]

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.]])

In [59]:
x[:, np.newaxis]

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.]])

In [60]:
x[np.newaxis, :]

array([[0., 1., 2., 3., 4.]])

In [61]:
x[np.newaxis, :] - x[:, np.newaxis]     #[5,1] - [1,5]

array([[ 0.,  1.,  2.,  3.,  4.],
       [-1.,  0.,  1.,  2.,  3.],
       [-2., -1.,  0.,  1.,  2.],
       [-3., -2., -1.,  0.,  1.],
       [-4., -3., -2., -1.,  0.]])

In [62]:
X[:,:].shape

(569, 30)

In [63]:
(X[:, None, :] - X[None, :, :]).shape

(569, 569, 30)

In [64]:
def distances_vectorized(X):
    """Compute pairwise distances between all instances (vectorized version).
    
    Args:
        X: data matrix, shape (N, D)
        
    Returns:
        dist: matrix of pairwise distances, shape (N, N)
    """
    return np.sqrt(((X[:, None] - X[None, :])**2).sum(-1))

In [65]:
dist2 = distances_vectorized(X)

Make sure that both variants produce the same results

In [66]:
# Direct comparison fails because of tiny numerical differences
np.all(dist1 == dist2)

True

In [67]:
# Two results are very close
np.linalg.norm(dist1 - dist2, ord='fro')

0.0

In [19]:
# Use np.allclose to compare
np.allclose(dist1, dist2)

True

### Best - library function

In [20]:
from scipy.spatial.distance import cdist, pdist, squareform

dist3 = cdist(X, X)
dist4 = squareform(pdist(X))

### Compare the runtime

In [21]:
%%timeit
dist1 = distances_for_loop(X)

3.28 s ± 53.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%%timeit
dist2 = distances_vectorized(X)

65.5 ms ± 247 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit
dist3 = cdist(X, X)

5.13 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit
dist4 = squareform(pdist(X))

2.91 ms ± 45.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Lessons:
1. For-loops are extremely slow! Avoid them whenever possible.
2. A better alternative - use matrix operations & broadcasting
3. An even better alternative - use library functions (if they are available).
4. Implementations with for-loops can be useful for debugging vectorized code.

# 2. Numerical stability
Typically, GPUs use single precision (32bit) floating point numbers (in some cases even half precision / 16bit). This significantly speeds ups the computations, but also makes numerical issues a lot more likely. 
Because of this we always have to be extremely careful to implement our code in a numerically stable way.

Most commonly, numerical issues occur when dealing with `log` and `exp` functions (e.g. when computing cross-entropy of a categorical distribution) and `sqrt` for values close to zero (e.g. when computing standard deviations or normalizing the $L_2$ norm).

## 2.1. Avoiding numerical overflow (exploding `exp`)

Softmax function $f : \mathbb{R}^D \to \Delta^{D - 1}$ converts a vector $\mathbf{x} \in \mathbb{R}^D$ into a vector of probabilities.

$$f(\mathbf{x})_j = \frac{\exp(x_j)}{\sum_{d=1}^{D} \exp(x_d)}$$

Apply the softmax function to the following vector.

In [25]:
x = np.linspace(0., 4., 5).astype(np.float32)
x

array([0., 1., 2., 3., 4.], dtype=float32)

In [26]:
# Our code here
denominator = np.exp(x).sum()
np.exp(x) / denominator

array([0.01165623, 0.03168492, 0.08612855, 0.23412167, 0.6364086 ],
      dtype=float32)

Now apply it to the following vector

In [27]:
x = np.linspace(50., 90., 5).astype(np.float32)
x

array([50., 60., 70., 80., 90.], dtype=float32)

In [28]:
# Our code here
denominator = np.exp(x).sum()
np.exp(x) / denominator

/home/shchur/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/home/shchur/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in exp
  This is separate from the ipykernel package so we can avoid doing imports until
/home/shchur/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([ 0.,  0.,  0.,  0., nan], dtype=float32)

How to avoid the exposion?

In [29]:
x_shifted = x - np.max(x)
denominator = np.exp(x_shifted).sum()
np.exp(x_shifted) / denominator

array([4.248161e-18, 9.357198e-14, 2.061060e-09, 4.539787e-05,
       9.999546e-01], dtype=float32)

## 2.2. Working in the log-space / simplifying the expressions

Binary cross entropy (BCE) loss for a logistic regression model (corresponds to negative log-likelihood of a Bernoulli model)

$$\log p(\mathbf{y} \mid \mathbf{X}, \mathbf{w}, b) = -\sum_{i=1}^{N} y_i \log \sigma(\mathbf{w}^T \mathbf{x}_i + b) + (1 - y_i) \log (1 - \sigma(\mathbf{w}^T \mathbf{x}_i + b))$$


Implement the BCE computation.

In [68]:
# TODO
def sigmoid(t):
    return 1 / (1 + np.exp(-t))

def binary_cross_entropy_unstable(scores, labels):
    return -labels * np.log(sigmoid(scores)) - (1 - labels) * np.log(1 - sigmoid(scores))

In [69]:
x = np.array([[20., 20.]])
w = np.array([[1., 1.]])
y = np.array([1.])

scores = x @ w.T
binary_cross_entropy_unstable(scores, y)

C:\Users\asus\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  
C:\Users\asus\anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in multiply
  


array([[nan]])

In [75]:
scores = x @ w.T
sigmoid(scores)

array([[1.]])

Try to simplify the BCE loss as much as possible

In [32]:
# TODO
def binary_cross_entropy_stable(scores, labels):
    return np.log(1 + np.exp(scores)) - labels * scores

binary_cross_entropy_stable(scores, y)

array([[0.]])

## 2.3. Loss of numerical precision

Implement the log sigmoid function 

$$f(x) = \log \sigma(x) = \log \left(\frac{1}{1 + \exp(-x)}\right)$$

In [33]:
# Your code here
def log_sigmoid_unstable(x):
    return np.log(1 / (1 + np.exp(-x)))

`float32` has much lower "resolution" than `float64`

In [34]:
x = np.linspace(0, 30, 11).astype(np.float32)
log_sigmoid_unstable(x)

array([-6.9314718e-01, -4.8587345e-02, -2.4756414e-03, -1.2338923e-04,
       -6.1989022e-06, -3.5762793e-07,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32)

In [35]:
x = np.linspace(0, 30, 11).astype(np.float64)
log_sigmoid_unstable(x)

array([-6.93147181e-01, -4.85873516e-02, -2.47568514e-03, -1.23402190e-04,
       -6.14419348e-06, -3.05902274e-07, -1.52299796e-08, -7.58256125e-10,
       -3.77513576e-11, -1.87960758e-12, -9.34807787e-14])

Implement the log-sigmoid function in a numerically stable way

In [36]:
def log_sigmoid_stable(x):
    return -np.log1p(np.exp(-x))

In [37]:
x = np.linspace(0, 30, 11).astype(np.float32)
log_sigmoid_stable(x)

array([-6.9314718e-01, -4.8587352e-02, -2.4756852e-03, -1.2340219e-04,
       -6.1441933e-06, -3.0590226e-07, -1.5229979e-08, -7.5825607e-10,
       -3.7751344e-11, -1.8795289e-12, -9.3576229e-14], dtype=float32)

Relevant functions: `np.log1p`, `np.expm1`, `scipy.special.logsumexp`, `scipy.special.softmax` -- these are also implemented in all major deep learning frameworks.

## Lessons:
1. Be especially careful when working with `log` and `exp` functions in **single precision** floating point arithmetics
2. Work in the log-space when possible
3. Use numerically stable library functions when available